## Metadata VDB Search
We will use Milvus and Towhee to help searches. Towhee is used to extract the semantics of the text and return the text embedding. The Milvus vector database can store and search vectors, and return related dataset's metadata. So we first need to install [Milvus](https://github.com/milvus-io/milvus) and [Towhee](https://github.com/towhee-io/towhee).

Before getting started, please make sure that you have started a [Milvus service](https://milvus.io/docs/install_standalone-docker.md). This notebook uses [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.2.11](https://milvus.io/docs/release_notes.md#2210).

### Package installations

In [1]:
#! pip install --upgrade pip
#! pip3 install -q towhee pymilvus==2.2.11
#! pip3 uninstall pymilvus -y

! pip3 install -q towhee pymilvus==2.1.1
! pip3 show pymilvus | grep -Ei 'Name:|Version:'
! pip3 show towhee | grep -Ei 'Name:|Version:'

Name: pymilvus
Version: 2.1.1
Name: towhee
Version: 1.1.3


## 1.1 Adding embeddings for columns

The dataset is from the [Kartverket dataset metadata](https://cdn.discordapp.com/attachments/1204433663035449384/1206537816654356480/metadata_no_format.csv?ex=65dc5ee7&is=65c9e9e7&hm=3b9a88db41103ef5393294c5eaeebb60ee2229f43724cc014d4cffc92de1f384&), which contains metadata about each dataset.

The strings in the columns need to be converted to vector representations (embedding) using Towhee [text_embedding.dpr operator](https://towhee.io/text-embedding/dpr). Columns containing these new embedings should contain the original column name with `_vector` at the end.

## 1.2 Load dataset and vectorise chosen column

In [3]:
import pandas as pd
from towhee import pipe, ops, DataCollection
from tqdm import tqdm


# Function to compute embeddings for a single text
def compute_embeddings(text):
    MAX_TOKENS = 512 # Temporary limit on number characters to convert
    truncated_text = text[:MAX_TOKENS]
    return DataCollection(embeddings_pipe(truncated_text)).to_list()[0]['vec']


# Loads dataset into dataframe and recasts columns into correct datatypes
df_kartverket = pd.read_excel('Metadata_excel.xlsx')
recast_to_string = ['datasetcreationdate', 'metadatacreationdate']
df_kartverket[recast_to_string] = df_kartverket[recast_to_string].astype('object')

# Fill NaN values with an empty string
df_kartverket.fillna('', inplace=True)

# Pipe converting text to embeddings (vectors)
embeddings_pipe = (
    pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .output('vec')
)

# Process each column and create new columns for embeddings
column_to_vectorise = 'title'
df_kartverket[column_to_vectorise + '_vector'] = df_kartverket[column_to_vectorise].apply(compute_embeddings)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## 2. Creation of Milvus collection for metadata

In [4]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

server_host = 'ebjerk.no'
server_port = '19530'

connections.connect(host=server_host, port=server_port)

def kartverket_create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
            #FieldSchema(name='schema', dtype=DataType.STRING, max_length=100), # REQUIRES STRING TYPE 
            FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=False),
            FieldSchema(name='uuid', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE
            #FieldSchema(name='hierarchyLevel', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE   
            #FieldSchema(name='hierarchyLevel_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), #REQUIRES STRING TYPE   
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=100),   
            FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=dim),

            #FieldSchema(name='datasetcreationdate', dtype=DataType.VARCHAR, max_length=500), # REQUIRES STRING TYPE   
            FieldSchema(name='abstract', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='abstract_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='keyword', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='keyword_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            #FieldSchema(name='geoBox', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE   
            #FieldSchema(name='geoBox_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), # REQUIRES STRING TYPE   
            FieldSchema(name='Constraints', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='Constraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            FieldSchema(name='SecurityConstraints', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='SecurityConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='LegalConstraints', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='LegalConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            #FieldSchema(name='temporalExtent', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE   
            ##FieldSchema(name='temporalExtent_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), # REQUIRES STRING TYPE   
            #FieldSchema(name='image', dtype=DataType.VARCHAR, max_length=1000), # REQUIRES STRING TYPE   
            FieldSchema(name='responsibleParty', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='responsibleParty_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            #FieldSchema(name='link', dtype=DataType.VARCHAR, max_length=500), # REQUIRES STRING TYPE   
            #FieldSchema(name='metadatacreationdate', dtype=DataType.VARCHAR, max_length=500), # REQUIRES STRING TYPE   
            ##FieldSchema(name='metadatacreationdate_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), # REQUIRES STRING TYPE   
            FieldSchema(name='productInformation', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='productInformation_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='parentId', dtype=DataType.VARCHAR, max_length=100),   
    ]
    schema = CollectionSchema(fields=fields, description='search text')
    collection = Collection(name=collection_name, schema=schema)
    
    index_params = {
        'metric_type': "L2",
        'index_type': "IVF_FLAT",
        'params': {"nlist": 2048}
    }
    collection.create_index(field_name='title_vector', index_params=index_params)
    return collection

kartverket_collection = kartverket_create_milvus_collection('kartverket_metadata', 768)

### 2.1 Creation of dataframe subset to exclude columns with complex data types

In [8]:
df_kartverket_slice = df_kartverket[['id', 'uuid', 'title', 'title_vector', 'abstract', 'keyword', 'Constraints', 'SecurityConstraints', 'LegalConstraints', 'responsibleParty', 'productInformation', 'parentId']]
df_kartverket_slice

,id,uuid,title,title_vector,abstract,keyword,Constraints,SecurityConstraints,LegalConstraints,responsibleParty,productInformation,parentId
0,37228,7a62f16f-9aeb-4c39-bf5f-e710232fa366,Artsfunn,"[0.56139773, 0.34755126, 0.14640424, 0.1977914...",Datasettet inneholder stedfestet informasjon a...,Natur###Norge###Svalbard###lav###karplanter###...,###,#########,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Frank HansenNorsk institutt for naturforskning...,Produktspesifikasjon###Produktark###Produktsid...,
1,21400,79013154-92ee-4647-b160-925cbc148601,Hav og is - Iskart (shapefil),"[0.03226918, 0.60322803, 0.5526991, -0.4393136...",Istjenesten ved Meteorologisk institutt utarbe...,Oceanographic geographical features###Inspire#...,Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Meteorologisk instituttistjenesten@met.no###Me...,Produktspesifikasjon###Produktark###Produktsid...,
2,240,f0083871-0d21-44e2-945f-9de9ea94d484,Losbordingsfelt,"[-0.5165257, 0.32858214, 0.57894915, -0.333858...",Bordingsfelt er angitt som et geografisk punkt...,Åpne data###Norge digitalt###modellbaserteVegp...,###,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Stian AamotKystverket37019700Kystveien 30Arend...,https://register.geonorge.no/register/versjone...,
3,21273,e379ef5e-8851-4305-b900-44a4587cf14c,Radnett - doseratemålestasjoner,"[0.087247156, 0.4677642, 0.1239377, -0.3505301...",Datasettet inneholder strålevernets radnettsta...,Norge digitalt###Åpne data###modellbaserteVegp...,Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Statens strålevernnrpa@nrpa.no###Direktoratet ...,Produktspesifikasjon###Produktark###Produktsid...,
4,37251,41ccca92-2ae9-43c9-9a45-b3d6424d1633,Predikert utbredelse og tetthetsfordeling av s...,"[-0.14394166, 0.41232172, -0.1769793, -0.33108...",Basert på gamle og nye data for forekomst av s...,Species distribution###Norge digitalt###modell...,Bruksbegrensninger Ingen ###IngenNo conditions...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Norsk institutt for naturforskningfrank.hansse...,Produktspesifikasjon###Produktark###Produktsid...,
...,...,...,...,...,...,...,...,...,...,...,...,...
184,55086,65509eba-5301-473c-8a91-84b2bba59173,Sentrumssoner 2016,"[-0.5731087, 0.2514926, 0.21160778, -0.3906372...",Inneholder avgrensning av og statistikk knytte...,Land use###Norge digitalt###geodataloven###fel...,###,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Erik EngelienStatistisk sentralbyråerik.engeli...,http://www.ssb.no/natur-og-miljo/_attachment/1...,55b401f3-2ea1-4045-9f87-22ac9d6ecf66
185,152,75461f65-eaea-4495-b2ab-3bc04d261669,Statistiske enheter grunnkretser - historiske ...,"[-0.11083123, -0.0798243, 0.12608653, 0.422164...",Datasettet viser grunnkretsinndelingen i Norge...,Statistical units###geodataloven###Norge digit...,Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Carina Tolpinrud JøntvedtKartverket+47 08700po...,https://register.geonorge.no/register/versjone...,02b6c97b-63da-4d46-9a70-6e9ef3442d54
186,55061,1d267604-5275-4c7a-91e4-9135f7e22cfb,Boligstatistikk på rutenett 250m 2019,"[-0.5389366, 0.28145486, 0.6831738, -0.2194134...",Inneholder rutenettstatistikk over antall boli...,Buildings###Norge digitalt###geodataloven###fe...,Bruksbegrensninger Ingen begrensninger på bruk...,Sikkerhetsnivå Ugradert: Available for general...,Tilgangsrestriksjoner Andre restriksjoner: Lim...,Statistisk Sentralbyråper.morten.holt@ssb.no##...,Produktspesifikasjon###https://register.geonor...,f3cdcd1f-5ee7-40fe-ac19-33a9101e00a4
187,50171,982c1c55-b96c-4021-aad1-a4a2d1bdbdee,Ar

### 2.2 Insert the subset dataframe data into Milvus collection

In [9]:
from towhee import ops, pipe, DataCollection

insert_pipe_kartverket = (pipe.input('df_kartverket_slice')
                   .flat_map('df_kartverket_slice', 'data', lambda df: df.values.tolist())
                   .map('data', 'res', ops.ann_insert.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name='kartverket_metadata'))
                   .output('res')
)

%time _ = insert_pipe_kartverket(df_kartverket_slice)


kartverket_collection.load()
kartverket_collection.num_entities

CPU times: user 598 ms, sys: 69.1 ms, total: 667 ms
Wall time: 6.84 s


189

## 3. Query against metadata collection

In [7]:
import numpy as np
# Variables specifying what column and collection to perform ANN comparrison against
vector_columns = ['title_vector']
collection_name = 'kartverket_metadata'

print(df_kartverket.columns)
# What columns to return for view
response_output = [
       'id', 'title',
       'abstract', 'keyword', 'Constraints',
       'SecurityConstraints', 'LegalConstraints',
       'responsibleParty', 'productInformation', 'parentId'
]


demo_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'title',
       'abstract', 'keyword', 'Constraints',
       'SecurityConstraints', 'LegalConstraints',
       'responsibleParty', 'productInformation', 'parentId'), 
                                       ops.ann_search.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name=collection_name,
                                                                    vector_field=vector_columns,
                                                                    output_fields=response_output, 
                                                                    limit=5))  
                    .output(*['query', 'score'], *response_output)
               )

kartverket_question1 = 'Just do it'
print(f'\n"{kartverket_question1}" search:')
res_kartverket1 = demo_pipe(kartverket_question1)
DataCollection(res_kartverket1).show()

Index(['schema', 'uuid', 'id', 'hierarchyLevel', 'title',
       'datasetcreationdate', 'abstract', 'keyword', 'geoBox', 'Constraints',
       'SecurityConstraints', 'LegalConstraints', 'temporalExtent', 'image',
       'responsibleParty', 'link', 'metadatacreationdate',
       'productInformation', 'parentId', 'title_vector'],
      dtype='object')

"Just do it" search:


query,score,id,title,abstract,keyword,Constraints,SecurityConstraints,LegalConstraints,responsibleParty,productInformation,parentId
